In [18]:
# https://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/PubTator/index.cgi?user=User622505252

In [1]:
from multiprocessing import Pool

In [2]:
num_cores = 4

In [3]:
chem = pd.read_csv('/Users/grace/workspace/bio_dataset/chemical2pubtator/chemical2pubtator', sep='\t')

In [4]:
chem.head()

,PMID,MeshID,Mentions,Resource
0,1,MESH:D000432,methanol,tmChem|MESH
1,1,MESH:D005561,Formate,MESH
2,10,MESH:D004074,Digitoxin,MESH
3,1000,MESH:D009249,NADP,MESH
4,1000001,MESH:D002118,calcium,tmChem|MESH


### 전체 목록 28411962

In [5]:
chem.shape

(28411962, 4)

In [6]:
chem.dtypes

PMID         int64
MeshID      object
Mentions    object
Resource    object
dtype: object

In [7]:
chem.loc[0]

PMID                   1
MeshID      MESH:D000432
Mentions        methanol
Resource     tmChem|MESH
Name: 0, dtype: object

In [8]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_cores)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [9]:
def convert_to_str(data):
    data['Resource'] = data['Resource'].apply(lambda x: str(x))
    data['Mentions'] = data['Mentions'].apply(lambda x: str(x))
    return data

In [10]:
%time chem = parallelize_dataframe(chem, convert_to_str)

CPU times: user 14.8 s, sys: 17 s, total: 31.8 s
Wall time: 39.4 s


### MeSH 만 12213942

In [11]:
chem = chem[chem.Resource.str.lower().str.find('mesh')>=0]

In [12]:
chem.shape

(12213942, 4)

### Unique 한 것만 396668

In [13]:
chem.drop_duplicates(subset=['Mentions'], inplace=True)

In [14]:
chem.shape

(396668, 4)

## 길이가 너무 짧은 약물 제외

In [15]:
chem = chem[chem.Mentions.str.len()>4]

In [16]:
chem.shape

(394342, 4)

In [17]:
chem.to_csv('./mesh_only.tsv', sep='\t')